In [3]:
##### import pickle
import os
import random
from datetime import datetime
import numpy as np

# =====================
# File Paths
# =====================
ACCOUNTS_FILE = "accounts.pkl"
TRANSACTIONS_FILE = "transactions.pkl"
CREDENTIALS_FILE = "credentials.pkl"

# =====================
# Data Storage
# =====================
accounts = {}
transactions = []
credentials = {}  # username -> (password, account_number)
current_user = None  # Track logged in user

# =====================
# BankAccount Class
# =====================
class BankAccount:
    def __init__(self, name, account_number, acc_type, balance=0):
        self.name = name
        self.account_number = account_number
        self.acc_type = acc_type
        self.balance = balance
        self.transactions = []

    def deposit(self, amount):
        if amount <= 0:
            print("Invalid deposit amount.")
            return
        self.balance += amount
        record = (self.account_number, datetime.now(), "Deposit", amount)
        self.transactions.append(record)
        append_transaction(record)

    def withdraw(self, amount):
        if amount <= 0:
            print("Invalid withdrawal amount.")
            return False
        if self.balance >= amount:
            self.balance -= amount
            record = (self.account_number, datetime.now(), "Withdraw", amount)
            self.transactions.append(record)
            append_transaction(record)
            return True
        else:
            return False

    def transfer(self, target_acc, amount):
        if amount <= 0:
            print("Invalid transfer amount.")
            return False
        if self.balance >= amount:
            self.withdraw(amount)
            target_acc.deposit(amount)
            record1 = (self.account_number, datetime.now(), f"Transfer to {target_acc.account_number}", amount)
            record2 = (target_acc.account_number, datetime.now(), f"Transfer from {self.account_number}", amount)
            append_transaction(record1)
            append_transaction(record2)
            return True
        return False

# =====================
# File Handling
# =====================
def save_data():
    with open(ACCOUNTS_FILE, "wb") as f:
        pickle.dump(accounts, f)
    with open(CREDENTIALS_FILE, "wb") as f:
        pickle.dump(credentials, f)

def load_data():
    global accounts, transactions, credentials
    if os.path.exists(ACCOUNTS_FILE):
        with open(ACCOUNTS_FILE, "rb") as f:
            accounts = pickle.load(f)
    if os.path.exists(TRANSACTIONS_FILE):
        with open(TRANSACTIONS_FILE, "rb") as f:
            try:
                while True:
                    transactions.append(pickle.load(f))
            except EOFError:
                pass
    if os.path.exists(CREDENTIALS_FILE):
        with open(CREDENTIALS_FILE, "rb") as f:
            credentials = pickle.load(f)

def append_transaction(transaction):
    with open(TRANSACTIONS_FILE, "ab") as f:
        pickle.dump(transaction, f)
    transactions.append(transaction)

# =====================
# Account Management
# =====================
def open_account():
    username = input("Choose a username: ")
    if username in credentials:
        print("Username already exists.")
        return
    password = input("Choose a password: ")

    # Password validation
    if len(password) < 6 or not any(c.isdigit() for c in password) or not any(c.isupper() for c in password):
        print("Password must be at least 6 characters, include a number and an uppercase letter.")
        return

    name = input("Enter account holder's name: ")
    acc_type = input("Enter account type (Savings/Current): ")
    initial_deposit = float(input("Enter initial deposit: "))
    account_number = random.randint(10000000, 99999999)

    account = BankAccount(name, account_number, acc_type, initial_deposit)
    accounts[account_number] = account
    credentials[username] = (password, account_number)
    print(f"Account created successfully! Account Number: {account_number}")
    save_data()

def login():
    global current_user
    username = input("Enter username: ")
    password = input("Enter password: ")
    if username in credentials and credentials[username][0] == password:
        current_user = username
        print(f"Login successful! Welcome, {username}")
    else:
        print("Invalid credentials.")

def logout():
    global current_user
    current_user = None
    print("Logged out successfully.")

def get_logged_in_account():
    if not current_user:
        print("You must log in first.")
        return None
    acc_num = credentials[current_user][1]
    return accounts[acc_num]

def view_account():
    acc = get_logged_in_account()
    if acc:
        print("\nAccount Details")
        print(f"Name: {acc.name}")
        print(f"Account Number: {acc.account_number}")
        print(f"Type: {acc.acc_type}")
        print(f"Balance: {acc.balance}\n")

# =====================
# Transactions
# =====================
def deposit():
    acc = get_logged_in_account()
    if acc:
        amount = float(input("Deposit amount: "))
        acc.deposit(amount)
        print("Deposit successful.")
        save_data()

def withdraw():
    acc = get_logged_in_account()
    if acc:
        amount = float(input("Withdrawal amount: "))
        if acc.withdraw(amount):
            print("Withdrawal successful.")
            save_data()
        else:
            print("Insufficient balance.")

def transfer():
    acc = get_logged_in_account()
    if acc:
        to_acc_num = int(input("Enter receiver's account number: "))
        amount = float(input("Enter amount to transfer: "))
        if to_acc_num in accounts:
            if acc.transfer(accounts[to_acc_num], amount):
                print("Transfer successful.")
                save_data()
            else:
                print("Insufficient balance.")
        else:
            print("Receiver account not found.")

# =====================
# Reports
# =====================
def view_transactions():
    acc_num = int(input("Enter account number: "))
    if acc_num not in accounts:
        print("Account not found.")
        return

    print(f"\n====== Transaction History for Account: {acc_num} ======\n")
    print(f"{'Date':<20} {'Type':<20} {'Amount':<10} {'Balance':<10}")
    print("-" * 60)

    account = accounts[acc_num]
    for t in account.transactions:
        date_str = t[1].strftime("%Y-%m-%d %H:%M:%S") if isinstance(t[1], datetime) else str(t[1])
        tran_type = t[2]
        amount = t[3]
        balance = account.balance if tran_type.startswith("Deposit") or tran_type.startswith("Transfer from") else account.balance  
        print(f"{date_str:<20} {tran_type:<20} {amount:<10.2f} {balance:<10.2f}")

    # Summary with numpy
    amounts = [t[3] for t in account.transactions]
    if amounts:
        deposits = [amt for (_, _, typ, amt) in account.transactions if "Deposit" in typ or "Transfer from" in typ]
        withdrawals = [amt for (_, _, typ, amt) in account.transactions if "Withdraw" in typ or "Transfer to" in typ]

        print("\n====== Account Summary ======")
        print(f"Total Transactions   : {len(amounts)}")
        print(f"Total Deposits       : {np.sum(deposits):.2f}")
        print(f"Total Withdrawals    : {np.sum(withdrawals):.2f}")
        print(f"Average Transaction  : {np.mean(amounts):.2f}")
        print(f"Current Balance      : {account.balance:.2f}")

# =====================
# Menu
# =====================
def menu():
    load_data()
    while True:
        print("\n====== Bank Menu ======")
        print("1. Open a new account")
        print("2. Login")
        print("3. View account details")
        print("4. Deposit")
        print("5. Withdraw")
        print("6. Transfer")
        print("7. View transaction history")
        print("8. Logout")
        print("9. Exit")
        choice = input("Choose option: ")

        if choice == "1": open_account()
        elif choice == "2": login()
        elif choice == "3": view_account()
        elif choice == "4": deposit()
        elif choice == "5": withdraw()
        elif choice == "6": transfer()
        elif choice == "7": view_transactions()
        elif choice == "8": logout()
        elif choice == "9":
            save_data()
            print("Exiting. Thank you!")
            break
        else:
            print("Invalid choice. Try again.")

# =====================
# Run
# =====================
menu()



====== Bank Menu ======
1. Open a new account
2. Login
3. View account details
4. Deposit
5. Withdraw
6. Transfer
7. View transaction history
8. Logout
9. Exit


Choose option:  7
Enter account number:  96372374



====== Transaction History for Account: 96372374 ======

Date                 Type                 Amount     Balance   
------------------------------------------------------------
2025-09-02 16:04:15  Deposit              1200.00    11200.00  

====== Account Summary ======
Total Transactions   : 1
Total Deposits       : 1200.00
Total Withdrawals    : 0.00
Average Transaction  : 1200.00
Current Balance      : 11200.00

====== Bank Menu ======
1. Open a new account
2. Login
3. View account details
4. Deposit
5. Withdraw
6. Transfer
7. View transaction history
8. Logout
9. Exit


Choose option:  3


You must log in first.

====== Bank Menu ======
1. Open a new account
2. Login
3. View account details
4. Deposit
5. Withdraw
6. Transfer
7. View transaction history
8. Logout
9. Exit


...


====== Transaction History for Account: 77958091 ======

Date                 Type                 Amount     Balance   
------------------------------------------------------------
2025-09-02 16:03:01  Deposit              15000.00   21800.00  
2025-09-02 16:03:21  Withdraw             2000.00    21800.00  
2025-09-02 16:04:15  Withdraw             1200.00    21800.00  

====== Account Summary ======
Total Transactions   : 3
Total Deposits       : 15000.00
Total Withdrawals    : 3200.00
Average Transaction  : 6066.67
Current Balance      : 21800.00

====== Bank Menu ======
1. Open a new account
2. Login
3. View account details
4. Deposit
5. Withdraw
6. Transfer
7. View transaction history
8. Logout
9. Exit


Choose option:  8


Logged out successfully.

====== Bank Menu ======
1. Open a new account
2. Login
3. View account details
4. Deposit
5. Withdraw
6. Transfer
7. View transaction history
8. Logout
9. Exit


Choose option:  9


Exiting. Thank you!
